# Modelo Relacional

Categoria(_id_, nome)

Regiao(_id_, nome)

Grupo(_id_, nome)

Receita(_id_, titulo, fonte, instrucoes, regiao_id, categoria_id)
  regiao_id chave estrangeira -> Regiao(id)
  categoria_id chave estrangeira -> Categoria(id)

Ingrediente(_id_, nome, grupo_id)
  grupo_id chave estrangeira -> Grupo(id)

ReceitaIngrediente(_receita_id_, _ingrediente_id_, qtde)
  _receita_id_ chave estrangeira -> Receita(id)
  _ingrediente_id_ chave estrangeira -> Ingrediente(id)

Nutriente(_id_, nome)

IngredienteNutriente(_ingrediente_id_, _nutriente_id_, valor, min, max)
  _ingrediente_id_ chave estrangeira -> Ingrediente(id)
  _nutriente_id_ chave estrangeira -> Nutriente(id)

ComponenteQuimico(_id_, nome, formula)

IngredienteComponenteQuimico(_ingrediente_id_, _componenteQuimico_id_)
  _ingrediente_id_ chave estrangeira -> Ingrediente(id)
  _componenteQuimico_id_ chave estrangeira -> ComponenteQuimico(id)


In [1]:
%defaultDatasource jdbc:h2:mem:db

In [11]:
DROP TABLE IF EXISTS Categoria;
DROP TABLE IF EXISTS Regiao;
DROP TABLE IF EXISTS Grupo;
DROP TABLE IF EXISTS Receita;
DROP TABLE IF EXISTS Ingrediente;
DROP TABLE IF EXISTS ReceitaIngrediente;
DROP TABLE IF EXISTS Nutriente;
DROP TABLE IF EXISTS IngredienteNutriente;
DROP TABLE IF EXISTS ComponenteQuimico;
DROP TABLE IF EXISTS IngredienteComponenteQuimico;

CREATE TABLE Categoria (
  id INT PRIMARY KEY,
  nome VARCHAR(255) NOT NULL
) AS SELECT
  Id, Name
FROM CSVREAD('p_categories.csv');


CREATE TABLE Regiao (
  id INT PRIMARY KEY,
  nome VARCHAR(255) NOT NULL
) AS SELECT
  Id, Name
FROM CSVREAD('p_regions.csv');


CREATE TABLE Grupo (
  id INT PRIMARY KEY,
  nome VARCHAR(255),
) AS SELECT
  id, description_group
FROM CSVREAD('p_groups.csv');


CREATE TABLE Receita (
  id INT PRIMARY KEY,
  titulo VARCHAR(255) NOT NULL,
  categoria_id INT,
  regiao_id INT,
  FOREIGN KEY (regiao_id) REFERENCES Regiao(id),
  FOREIGN KEY (categoria_id) REFERENCES Categoria(id)
) AS SELECT
  id_recipe, name_recipe, category_id, region_id
FROM CSVREAD('p_recipes.csv');


CREATE TABLE Ingrediente (
  id INT PRIMARY KEY,
  nome VARCHAR(255) NOT NULL,
  description CHAR, 
  grupo_id INT,
  FOREIGN KEY (grupo_id) REFERENCES Grupo(id)
) AS SELECT
  id, name, description, group_id
FROM CSVREAD('p_ingredients.csv');


CREATE TABLE ReceitaIngrediente (
  ingrediente_id INT,
  receita_id INT,
  qtde Varchar(50),
  FOREIGN KEY (receita_id) REFERENCES Receita(id),
  FOREIGN KEY (ingrediente_id) REFERENCES Ingrediente(id)
) AS SELECT
   id_ingredient, id_recipe, qtde
FROM CSVREAD('p_ingredients_recipes.csv');


CREATE TABLE Nutriente (
  id INT PRIMARY KEY,
  public_id VARCHAR(255),
  legacy_id INT,
  nome VARCHAR(255) NOT NULL
) AS SELECT
  nutrient_id, public_id, legacy_id, name
FROM CSVREAD('p_nutrients.csv');


CREATE TABLE IngredienteNutriente (
  ingrediente_id INT,
  nutriente_id INT,
  valor DOUBLE,
  min DOUBLE,
  max DOUBLE,
  FOREIGN KEY (ingrediente_id) REFERENCES Ingrediente(id),
  FOREIGN KEY (nutriente_id) REFERENCES Nutriente(id)
) AS SELECT
  id_ingredient, id_nutrient, value, min, max
FROM CSVREAD('p_ingredients_nutrients.csv');


CREATE TABLE ComponenteQuimico (
  id INT PRIMARY KEY,
  nome VARCHAR(255) NOT NULL,
  formula VARCHAR(255)
) AS SELECT
  compound_id, public_id, name
FROM CSVREAD('p_compounds.csv');


CREATE TABLE IngredienteComponenteQuimico (
  ingrediente_id INT,
  componenteQuimico_id INT,
  FOREIGN KEY (ingrediente_id) REFERENCES Ingrediente(id),
  FOREIGN KEY (componenteQuimico_id) REFERENCES ComponenteQuimico(id)
) AS SELECT
  id_ingredient, id_compound
FROM CSVREAD('p_ingredients_compounds2.csv');

In [12]:
-- Quais os ingredientes que compõem determinadas receitas?
SELECT r.titulo AS receita, i.nome AS ingrediente, ri.qtde
FROM ReceitaIngrediente ri
JOIN Receita r ON ri.receita_id = r.id
JOIN Ingrediente i ON ri.ingrediente_id = i.id
WHERE r.titulo = 'Sushi';

In [19]:
-- Quais os ingredientes mais consumidos e em cada região?
SELECT
  R.regiao_id,
  I.nome AS ingrediente,
  COUNT(*) AS total_consumido
FROM
  ReceitaIngrediente RI
  JOIN Receita R ON RI.receita_id = R.id
  JOIN Ingrediente I ON RI.ingrediente_id = I.id
GROUP BY
  R.regiao_id,
  I.nome
ORDER BY
  R.regiao_id,
  total_consumido DESC;

In [17]:
-- Quais os nutrientes mais consumidos em cada região?
SELECT r.regiao_id, n.nome AS nutriente, SUM(in.valor) AS quantidade_consumida
FROM IngredienteNutriente in
JOIN Nutriente n ON in.nutriente_id = n.idS
JOIN Ingrediente i ON in.ingrediente_id = i.id
JOIN ReceitaIngrediente ri ON i.id = ri.ingrediente_id
JOIN Receita r ON ri.receita_id = r.id
GROUP BY r.regiao_id, n.nome
ORDER BY quantidade_consumida DESC;

In [8]:
-- componentes químicos que mais aparecem em uma determinada categoria
SELECT
  cq.formula AS ComponenteQuimico,
  COUNT(icq.ingrediente_id) AS Quantidade
FROM IngredienteComponenteQuimico icq
JOIN Ingrediente i ON icq.ingrediente_id = i.id
JOIN Categoria c ON i.grupo_id = c.id
JOIN ComponenteQuimico cq ON icq.componenteQuimico_id = cq.id
WHERE c.id = 6
GROUP BY cq.formula
ORDER BY Quantidade DESC
LIMIT 10;
 

In [30]:
-- componentes químicos que menos aparecem em uma determinada categoria
SELECT
  cq.formula AS ComponenteQuimico,
  COUNT(icq.ingrediente_id) AS Quantidade
FROM IngredienteComponenteQuimico icq
JOIN Ingrediente i ON icq.ingrediente_id = i.id
JOIN Categoria c ON i.grupo_id = c.id
JOIN ComponenteQuimico cq ON icq.componenteQuimico_id = cq.id
WHERE c.id = 6
GROUP BY cq.formula
ORDER BY Quantidade ASC
LIMIT 10;